In [2]:
import zipfile
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('all')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, classification_report, recall_score
import warnings
from pandas.errors import SettingWithCopyWarning
import joblib
import tensorflow as tf
import os
import torch
from transformers import BertTokenizer, BertModel
import umap
import hdbscan
from sklearn.linear_model import LogisticRegression
from bertopic import BERTopic

uploaded_zip = '/home/abarovic/popeREU/eclipse_platform.zip'
extract_dir = '/home/abarovic/popeREU'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)
with zipfile.ZipFile(uploaded_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

csv_file = os.path.join(extract_dir, 'eclipse_platform.csv')
fulldata = pd.read_csv(csv_file)

print(fulldata.columns)
print(fulldata.isna().sum())  # Check NaNs in training features

def CreateDuration(dataset):
    dataset['Created_time'] = pd.to_datetime(dataset['Created_time'], utc=True)
    dataset['Resolved_time'] = pd.to_datetime(dataset['Resolved_time'], utc=True)

    # Convert to the desired format
    dataset['Created_time2'] = dataset['Created_time'].dt.strftime('%m/%d/%Y %H:%M')
    dataset['Resolved_time2'] = dataset['Resolved_time'].dt.strftime('%m/%d/%Y %H:%M')

    # Parse the formatted datetime strings back to datetime objects to ensure they are in the correct format
    dataset['Created_time2'] = pd.to_datetime(dataset['Created_time2'], format='%m/%d/%Y %H:%M')
    dataset['Resolved_time2'] = pd.to_datetime(dataset['Resolved_time2'], format='%m/%d/%Y %H:%M')

    # Calculate the duration in hours
    dataset['Duration'] = dataset['Resolved_time2'] - dataset['Created_time2']
    dataset['Duration_hours'] = dataset['Duration'].dt.total_seconds() / 3600

    # Drop the intermediate columns
    dataset.drop(['Created_time2', 'Resolved_time2', 'Duration'], axis=1, inplace=True)

CreateDuration(fulldata)

def NoOutliers(df):
    Q1 = df['Duration_hours'].quantile(0.25)
    Q3 = df['Duration_hours'].quantile(0.75)

    # Calculate the IQR
    IQR = Q3 - Q1

    # Define the lower and upper bounds for filtering outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out the outliers
    df = df[(df['Duration_hours'] >= lower_bound) & (df['Duration_hours'] <= upper_bound)]
    return(df)

#fulldata = NoOutliers(fulldata)

def CreateTimeLabel(df):
    threshold = df['Duration_hours'].quantile(0.70)
    # Create a new column TimeLabel based on the threshold
    df['TimeLabel'] = df['Duration_hours'].apply(lambda x: 'short' if x <= threshold else 'long')
    df = df[df['Resolution'] != 'MOVED']
    #df = df[df['Resolution'] == 'FIXED']
    return df

fulldata = CreateTimeLabel(fulldata)
def RemoveStopWords(dataset):
    #making all the items in the descriptions columns lower
    dataset['Description'] = dataset['Description'].str.lower()
    # Define stop words
    stop_words = set(stopwords.words('english'))
     # Remove stop words
    for index, row in dataset.iterrows():
        # Check if the 'Description' is not NaN
        if isinstance(row['Description'], str):
            words = row['Description'].split()
            final_tokens = [word for word in words if word not in stop_words]
            dataset.at[index, 'Description'] = ' '.join(final_tokens)
    return dataset

fulldata = RemoveStopWords(fulldata)

def Lemmitization(dataset):
    # Initialize the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Function to lemmatize a sentence
    def lemmatize_sentence(sentence):
        words = nltk.word_tokenize(sentence)
        lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]  # Lemmatize verbs (default)
        return ' '.join(lemmatized_words)
    dataset['Description'] = dataset['Description'].astype(str)  # Convert all to strings
    # Apply lemmatization to each row in the 'Description' column
    dataset['Description'] = dataset['Description'].apply(lemmatize_sentence)

Lemmitization(fulldata)

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None

# Function to calculate sentiment scores
def CalculateSentimentScores(description):
    tokens = word_tokenize(description)
    tagged_tokens = pos_tag(tokens)
        
    pos_score = 0
    neg_score = 0
    token_count = 0
    for word, tag in tagged_tokens:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
            continue
        
        synsets = list(swn.senti_synsets(word, wn_tag))
        if not synsets:
            continue
        
        # Use the first synset for simplicity
        synset = synsets[0]
        pos_score += synset.pos_score()
        neg_score += synset.neg_score()
        token_count += 1
    
    # Normalize scores by the number of tokens
    if token_count > 0:
        pos_score /= token_count
        neg_score /= token_count
        return pos_score, neg_score

# Apply sentiment score calculation to each description
def CreatePosNegColumns(dataset): 
    dataset[['Pos_Score', 'Neg_Score']] = dataset['Description'].apply(lambda x: pd.Series(CalculateSentimentScores(x)))

CreatePosNegColumns(fulldata)

def EmotionColumn(dataset):
    #Creating a column for binary emotion (positive or negative)
    dataset['Emotion'] = dataset['Pos_Score'] - dataset['Neg_Score']
    
    # Assign labels based on the difference
    dataset['Emotion'] = dataset['Emotion'].apply(lambda x: 'positive' if x > 0 else 'negative')

EmotionColumn(fulldata)

def EmotionalityColumn(dataset):
    dataset['Emotionality'] = dataset['Pos_Score']+ dataset['Neg_Score']

EmotionalityColumn(fulldata)

def Destiny(df):
    # Create a new column 'Destiny' with default value 'Not Fixed'
    df['Destiny'] = 'Not Fixed'
    # Update 'Destiny' based on conditions in 'Resolution' column
    df.loc[df['Resolution'] == 'FIXED', 'Destiny'] = 'Fixed'
    return df

fulldata = Destiny(fulldata)
#print(fulldata['Emotion'].head(100))

def Emotion(df):
    # Fill NaN values with a default value (e.g., 'neutral') before mapping
    df['Emotion'] = df['Emotion'].fillna('neutral')
    # Label Positive emotion as 1, negative as 0, and neutral as -1
    df['Emotion'] = df['Emotion'].map({'positive': 1, 'negative': 0, 'neutral': -1})
    return df

fulldata = Emotion(fulldata)


def Priority(df):
    # Fill NaN values with a default value (e.g., 'P5') before mapping
    df['Priority'] = df['Priority'].fillna('P5')
    # Label Priority Columns numerically
    df['Priority'] = df['Priority'].map({'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4, 'P5': 5})
    return df

fulldata = Priority(fulldata)

def DropDescriptionNA(df):
    df = df[df['Description'] != 'nan']

DropDescriptionNA(fulldata)

def Model(df):
    model = BERTopic(nr_topics=20)

    # Prepare the documents
    docs = df['Description'].fillna('').astype(str).tolist()

    # Fit the model to your data
    topics, probabilities = model.fit_transform(docs)
    #adding a column for predicted topics
    new_docs = df['Description'].fillna('').astype(str).to_list()
    topics, probs = model.transform(new_docs)
    df['Predicted_Topic'] = topics

    # Get topic information
    topic_info = model.get_topic_info()
    print(topic_info)
    # Get words associated with each topic
    topic_words = {topic: model.get_topic(topic) for topic in topic_info.Topic}
    print(topic_words)
    return df

fulldata = Model(fulldata)

print(fulldata.columns)

print(fulldata.columns)
print(fulldata['Emotionality'].head(100))

def SplitData(dataset):
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    trainset, testset = dataset[:train_size], dataset[train_size:]
    return trainset, testset
trainset, testset = SplitData(fulldata)
print("Total items in dataset:", fulldata.shape[0])
print("Total items in trainset:", trainset.shape[0])
print("Total items in testset:", testset.shape[0])

print(trainset.columns)
print(testset.columns)

print(trainset['Emotionality'].head(100))

print("-----------------\n Training Set: \n-----------------")
print(trainset.isna().sum())  # Check NaNs in training features
print("-----------------\n Test Set: \n-----------------")
print(testset.isna().sum())   # Check NaNs in test features


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]  

Index(['Issue_id', 'Priority', 'Component', 'Duplicated_issue', 'Title',
       'Description', 'Status', 'Resolution', 'Version', 'Created_time',
       'Resolved_time'],
      dtype='object')
Issue_id                0
Priority                0
Component               0
Duplicated_issue    70752
Title                   0
Description           129
Status                  0
Resolution              0
Version                 0
Created_time            0
Resolved_time           0
dtype: int64


/home/abarovic/anaconda3/envs/pope_env/lib/python3.8/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The curren

    Topic  Count                                               Name  \
0      -1  44004                           -1_file_eclipse_line_use   
1       0  32592                                  0_at_use_file_new   
2       1   3793                          1_progress_patch_job_file   
3       2    960                          2_search_filter_sort_task   
4       3    734                              3_note_w00t_dsgsg_den   
5       4    680                       4_file_ftp_filedialog_export   
6       5    641                        5_bind_value_object_dispose   
7       6    500                      6_attachment_create_none_note   
8       7    395                          7_history_undo_view_local   
9       8    134                       8_summary_attach_picture_say   
10      9    131                             9_nan_tycho_bogus_work   
11     10    127                      10_styledtext_line_text_caret   
12     11    113                      11_columns_column_resize_size   
13    

CNN (Emotion, Emotionality, priority)

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

trainset, testset = SplitData(fulldata)
# Define features and target
features = ['Emotion', 'Emotionality', 'Priority']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Define CNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (short vs long)
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Epoch 1/20


2025-02-27 21:29:58.380239: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


4258/4258 [==============================] - 13s 3ms/step - loss: 0.6073 - accuracy: 0.7039 - val_loss: 0.5348 - val_accuracy: 0.7904
Epoch 2/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6053 - accuracy: 0.7047 - val_loss: 0.5384 - val_accuracy: 0.7904
Epoch 3/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6043 - accuracy: 0.7048 - val_loss: 0.5453 - val_accuracy: 0.7903
Epoch 4/20
4258/4258 [==============================] - 12s 3ms/step - loss: 0.6041 - accuracy: 0.7047 - val_loss: 0.5403 - val_accuracy: 0.7903
Epoch 5/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6038 - accuracy: 0.7046 - val_loss: 0.5402 - val_accuracy: 0.7903
Epoch 6/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6036 - accuracy: 0.7046 - val_loss: 0.5436 - val_accuracy: 0.7903
Epoch 7/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6028 - accuracy: 0.7047 - val_loss: 0.5408 - val_accuracy: 0.7

CNN (Emotion, Emotionality, priority, BERT)

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

trainset, testset = SplitData(fulldata)
# Define features and target
features = ['Emotion', 'Emotionality', 'Priority', 'Predicted_Topic']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Define CNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (short vs long)
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Epoch 1/20
4258/4258 [==============================] - 14s 3ms/step - loss: 0.6077 - accuracy: 0.7036 - val_loss: 0.5368 - val_accuracy: 0.7903
Epoch 2/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6048 - accuracy: 0.7047 - val_loss: 0.5480 - val_accuracy: 0.7903
Epoch 3/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6045 - accuracy: 0.7047 - val_loss: 0.5443 - val_accuracy: 0.7903
Epoch 4/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6032 - accuracy: 0.7047 - val_loss: 0.5383 - val_accuracy: 0.7903
Epoch 5/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6029 - accuracy: 0.7047 - val_loss: 0.5347 - val_accuracy: 0.7903
Epoch 6/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6023 - accuracy: 0.7046 - val_loss: 0.5334 - val_accuracy: 0.7903
Epoch 7/20
4258/4258 [==============================] - 13s 3ms/step - loss: 0.6024 - accuracy: 0.7047 - val_loss: 0.5380 - val_ac

CNN (Emotion, emotionality, priority) -> Weighted

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from collections import Counter

trainset, testset = SplitData(fulldata)
# Define features and target
features = ['Emotion', 'Emotionality', 'Priority']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print("Resampled class distribution:", Counter(y_train))

# Define CNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (short vs long)
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Resampled class distribution: Counter({1: 46166, 0: 46166})
Epoch 1/20
5771/5771 [==============================] - 17s 3ms/step - loss: 0.6682 - accuracy: 0.5509 - val_loss: 0.6642 - val_accuracy: 0.6970
Epoch 2/20
5771/5771 [==============================] - 17s 3ms/step - loss: 0.6672 - accuracy: 0.5512 - val_loss: 0.6988 - val_accuracy: 0.4673
Epoch 3/20
5771/5771 [==============================] - 17s 3ms/step - loss: 0.6659 - accuracy: 0.5543 - val_loss: 0.6835 - val_accuracy: 0.6156
Epoch 4/20
5771/5771 [==============================] - 16s 3ms/step - loss: 0.6655 - accuracy: 0.5503 - val_loss: 0.6880 - val_accuracy: 0.5620
Epoch 5/20
5771/5771 [==============================] - 17s 3ms/step - loss: 0.6652 - accuracy: 0.5536 - val_loss: 0.6949 - val_accuracy: 0.5273
Epoch 6/20
5771/5771 [==============================] - 16s 3ms/step - loss: 0.6650 - accuracy: 0.5522 - val_loss: 0.6844 - val_accuracy: 0.5199
Epoch 7/20
5771/5771 [==============================] - 17s 3ms/step -

CNN (Emotion, emotionality, priority, BERT) -> Weighted

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from collections import Counter

trainset, testset = SplitData(fulldata)
# Define features and target
features = ['Emotion', 'Emotionality', 'Priority', 'Predicted_Topic']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print("Resampled class distribution:", Counter(y_train))


# Define CNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (short vs long)
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Resampled class distribution: Counter({1: 46166, 0: 46166})
Epoch 1/20
5771/5771 [==============================] - 15s 3ms/step - loss: 0.6687 - accuracy: 0.5488 - val_loss: 0.6916 - val_accuracy: 0.4822
Epoch 2/20
5771/5771 [==============================] - 16s 3ms/step - loss: 0.6669 - accuracy: 0.5522 - val_loss: 0.6737 - val_accuracy: 0.6370
Epoch 3/20
5771/5771 [==============================] - 17s 3ms/step - loss: 0.6654 - accuracy: 0.5525 - val_loss: 0.6680 - val_accuracy: 0.7027
Epoch 4/20
5771/5771 [==============================] - 17s 3ms/step - loss: 0.6651 - accuracy: 0.5531 - val_loss: 0.6748 - val_accuracy: 0.6173
Epoch 5/20
5771/5771 [==============================] - 16s 3ms/step - loss: 0.6650 - accuracy: 0.5529 - val_loss: 0.6906 - val_accuracy: 0.5531
Epoch 6/20
5771/5771 [==============================] - 17s 3ms/step - loss: 0.6649 - accuracy: 0.5530 - val_loss: 0.6743 - val_accuracy: 0.6542
Epoch 7/20
5771/5771 [==============================] - 17s 3ms/step -